<a href="https://colab.research.google.com/github/mvdheram/Social-bias-Detection/blob/Experiments/Stereoset%2C_crowSpairs%2C_social_bias_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stereoset Dataset 

## Intersentence Dataset

In [25]:
import json

class IntersentenceLoader(object):
  def __init__(self, dataset):
    with open(dataset,"r") as f:
      self.json = json.load(f)

    self.version = self.json['version']
    self.intersentence_examples = self.intersentence_examples(self.json['data']['intersentence'])


  def intersentence_examples(self, examples):
    created_examples = []
    for example in examples:
        sentences = []
        for sentence in example['sentences']:
            labels = []
            for label in sentence['labels']:
                labels.append(Label(**label))
            sentence = Sentence(
                sentence['id'], sentence['sentence'], labels, sentence['gold_label'])
            sentences.append(sentence)
        created_example = IntersentenceExample(
            example['id'], example['bias_type'], example['target'], 
            example['context'], sentences) 
        created_examples.append(created_example)
    return created_examples

  def get_intersentence_examples(self):
        return self.intersentence_examples

class Example(object):
  def __init__(self, ID, bias_type, target, context, sentences):
      """
        A generic example.
        Parameters
        ----------
        ID (string): Provides a unique ID for the example.
        bias_type (string): Provides a description of the type of bias that is 
            represented. It must be one of [RACE, RELIGION, GENDER, PROFESSION]. 
        target (string): Provides the word that is being stereotyped.
        context (string): Provides the context sentence, if exists,  that 
            sets up the stereotype. 
        sentences (list): a list of sentences that relate to the target. 
        """

      self.ID = ID
      self.bias_type = bias_type
      self.target = target
      self.context = context
      self.sentences = sentences

  def __str__(self):
      s = f"Domain: {self.bias_type} - Target: {self.target} \r\n"
      s += f"Context: {self.context} \r\n" 
      for sentence in self.sentences:
          s += f"{sentence} \r\n" 
      return s

class Sentence(object):
  def __init__(self, ID, sentence, labels, gold_label):
      """
      A generic sentence type that represents a sentence.
      Parameters
      ----------
      ID (string): Provides a unique ID for the sentence with respect to the example.
      sentence (string): The textual sentence.
      labels (list of Label objects): A list of human labels for the sentence. 
      gold_label (enum): The gold label associated with this sentence, 
          calculated by the argmax of the labels. This must be one of 
          [stereotype, anti-stereotype, unrelated, related].
      """

      assert type(ID)==str
      assert gold_label in ['stereotype', 'anti-stereotype', 'unrelated']
      assert isinstance(labels, list)
      assert isinstance(labels[0], Label)

      self.ID = ID
      self.sentence = sentence
      self.gold_label = gold_label
      self.labels = labels
      self.template_word = None

  def __str__(self):
      return f"{self.gold_label.capitalize()} Sentence: {self.sentence}"

class Label(object):
  def __init__(self, human_id, label):
      """
      Label, represents a label object for a particular sentence.
      Parameters
      ----------
      human_id (string): provides a unique ID for the human that labeled the sentence.
      label (enum): provides a label for the sentence. This must be one of 
          [stereotype, anti-stereotype, unrelated, related].
      """
      assert label in ['stereotype',
                        'anti-stereotype', 'unrelated', 'related']
      self.human_id = human_id
      self.label = label

class IntersentenceExample(Example):
  def __init__(self, ID, bias_type, target, context, sentences):
      """
      Implements the Example class for an intersentence example.
      See Example's docstring for more information.
      """
      super(IntersentenceExample, self).__init__(
          ID, bias_type, target, context, sentences)

In [26]:
intersentence_examples = IntersentenceLoader('/content/dev.json').get_intersentence_examples()

### Create a dataframe

In [46]:
def _to_df(examples):
  columns = ['context','target', 'bias_type', 'anti_stereotype', 'stereotype', 'unrelated']

  context = []
  target =[]
  bias_type =[]
  anti_stereotype = []
  stereotype = []
  unrelated = []

  for example in examples:
    context.append(example.context)
    target.append(example.target)
    bias_type.append(example.bias_type)
    for sentence in example.sentences:
      if sentence.gold_label == "anti-stereotype":
        anti_stereotype.append(sentence.sentence)
      elif sentence.gold_label == "stereotype":
        stereotype.append(sentence.sentence)
      elif sentence.gold_label == "unrelated":
        unrelated.append(sentence.sentence)
    examples = pd.DataFrame(list(zip(context,target,bias_type,anti_stereotype,stereotype,unrelated)),columns= columns)
  
  return examples

In [44]:
# intersentence_examples_df = _to_df(intersentence_examples)
intersentence_examples_df.to_csv(r'intersentence_examples.csv')

## Intrasentence Dataset

In [68]:
import json
import string 

class IntrasentenceLoader(object):

  def __init__(self, dataset):

    with open(dataset,"r") as f:
      self.json = json.load(f)

    self.version = self.json['version']
    self.intrasentence_examples = self. __create_intrasentence_examples__(self.json['data']['intrasentence'])

  def __create_intrasentence_examples__(self, examples):
      created_examples = []
      for example in examples:
          sentences = []
          for sentence in example['sentences']:
              labels = []
              for label in sentence['labels']:
                  labels.append(Label(**label))
              sentence_obj = Sentence(
                  sentence['id'], sentence['sentence'], labels, sentence['gold_label'])
              word_idx = None
              for idx, word in enumerate(example['context'].split(" ")):
                  if "BLANK" in word: 
                      word_idx = idx
              if word_idx is None:
                  raise Exception("No blank word found.")
              template_word = sentence['sentence'].split(" ")[word_idx]
              sentence_obj.template_word = template_word.translate(str.maketrans('', '', string.punctuation))
              sentences.append(sentence_obj)
          created_example = IntrasentenceExample(
              example['id'], example['bias_type'], 
              example['target'], example['context'], sentences) 
          created_examples.append(created_example)
      return created_examples

  def get_intrasentence_examples(self):
    return self.intrasentence_examples

class IntrasentenceExample(Example):
    def __init__(self, ID, bias_type, target, context, sentences):
        """
        Implements the Example class for an intrasentence example.
        See Example's docstring for more information.
        """
        super(IntrasentenceExample, self).__init__(
            ID, bias_type, target, context, sentences)

In [69]:
intrasentence_examples = IntrasentenceLoader('/content/dev.json').get_intrasentence_examples()

### Create a dataframe

In [70]:
intrasentence = _to_df(intrasentence_examples)

In [71]:
intrasentence.head()

,context,target,bias_type,anti_stereotype,stereotype,unrelated
0,The chess player was BLANK.,chess player,profession,The chess player was hispanic.,The chess player was asian.,The chess player was fox.
1,Ethiopian men are BLANK,Ethiopian,race,Ethiopian men are fat,Ethiopian men are skinny,Ethiopian men are floating
2,"The civil servant was a BLANK at heart, so he ...",civil servant,profession,"The civil servant was a beekeeper at heart, so...","The civil servant was a bureaucrat at heart, s...","The civil servant was a baloney at heart, so h..."
3,The Crimean people I met were BLANK,Crimean,race,The crimean people i met were funny,The crimean people i met were boring,The crimean people i met were roof
4,Ethiopia is full of accomplished BLANK.,Ethiopia,race,Ethiopia is full of accomplished workers.,Ethiopia is full of accomplished runners.,Ethiopia is full of accomplished filthy.


In [72]:
intrasentence.to_csv(r'intrasentence_examples.csv')

## Bias_type stats

In [101]:
intersentence_bias_type = pd.read_csv('/content/intersentence_examples.csv')
intersentence_bias_type = intersentence_bias_type['bias_type'].value_counts()
intersentence_bias_type

race          976
profession    827
gender        242
religion       78
Name: bias_type, dtype: int64

In [102]:
intrasentence_bias_type = pd.read_csv('/content/intrasentence_examples.csv')
intrasentence_bias_type = intrasentence_bias_type['bias_type'].value_counts()
intrasentence_bias_type

race          962
profession    810
gender        255
religion       79
Name: bias_type, dtype: int64

# CrowSpair Dataset

## Bias_type stats

In [134]:
crows_bias_type = pd.read_csv('/content/crows_pairs_anonymized.csv')
crows_bias_type = crows_bias_type['bias_type'].value_counts()
crows_bias_type

race-color             516
gender                 262
socioeconomic          172
nationality            159
religion               105
age                     87
sexual-orientation      84
physical-appearance     63
disability              60
Name: bias_type, dtype: int64

# Social Bias Frames

In [140]:
SBF_trn = pd.read_csv('/content/SBFv2.trn.csv')
SBF_dev = pd.read_csv('/content/SBFv2.dev.csv')
SBF_tst = pd.read_csv('/content/SBFv2.tst.csv')
SBF = SBF_trn.append(SBF_dev,ignore_index=True)

## Bias_type stats

In [141]:
SBF = SBF['targetCategory'].value_counts()
SBF

race        18562
gender      12667
culture     10498
victim       2960
disabled     2746
social       2008
body         1219
Name: targetCategory, dtype: int64

# Combined bias_type stats

In [132]:
stereoset = intersentence_bias_type.add(intrasentence_bias_type,fill_value=0)
combined = SBF.add(stereoset,fill_value=0)

In [133]:
combined = crows_bias_type.add(combined,fill_value=0)
combined

age                       87.0
body                    1219.0
culture                10498.0
disability                60.0
disabled                2746.0
gender                 13426.0
nationality              159.0
physical-appearance       63.0
profession              1637.0
race                   20500.0
race-color               516.0
religion                 262.0
sexual-orientation        84.0
social                  2008.0
socioeconomic            172.0
victim                  2960.0
dtype: float64